In [2]:
# -------------------------------------------
# Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, desc
from pyspark.sql.types import DoubleType, FloatType, IntegerType, LongType
import pyspark.sql.functions as F

# -------------------------------------------
# Crear sesión de Spark 
try:
    spark = SparkSession.builder.getOrCreate()
except:
    spark = SparkSession.builder \
        .appName("Exploracion_Completa_Dataset") \
        .config("spark.sql.shuffle.partitions", "100") \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# -------------------------------------------
# Cargar el dataset
nombre_archivo = "Books_rating.csv"  
df = spark.read.csv(nombre_archivo, header=True, inferSchema=True)

# Contar registros totales
num_total = df.count()

# -------------------------------------------
# Estadísticas Generales del Dataset

# Número de columnas
num_columnas = len(df.columns)
print(f"Número de columnas: {num_columnas}")

# Número de registros
print(f"Número de registros: {num_total}")

# Tipos de datos de cada columna
print("\nTipos de datos de cada columna:")
df.printSchema()

# Registros con al menos un valor faltante
condiciones_nulos = [col(c).isNull() | (col(c) == "") for c in df.columns]
df_nulos = df.withColumn("nulos", sum([when(cond, 1).otherwise(0) for cond in condiciones_nulos]))
total_registros_con_nulos = df_nulos.filter(col("nulos") > 0).count()
print(f"\nNúmero de registros con al menos un valor faltante: {total_registros_con_nulos}")

# Columnas con valores faltantes
# Columnas numéricas
numeric_columns = [field.name for field in df.schema.fields if isinstance(field.dataType, (DoubleType, FloatType))]

# Conteo de nulos
expressions = []
for c in df.columns:
    if c in numeric_columns:
        expressions.append(count(when(col(c).isNull() | isnan(c), c)).alias(c))
    else:
        expressions.append(count(when(col(c).isNull() | (col(c) == ""), c)).alias(c))

missing_counts = df.select(expressions).collect()[0].asDict()

# Filtrar columnas con nulos
columnas_con_faltantes = [columna for columna, nulos in missing_counts.items() if nulos > 0]

print("\nColumnas con valores faltantes:")
for columna in columnas_con_faltantes:
    print(f"- {columna}")

# -------------------------------------------
# Análisis detallado por columna
def analizar_columna(df, columna, num_total):
    print(f"\n--- Columna: {columna} ---")
    col_data = df.select(columna)
    
    # Valores nulos
    if col_data.schema.fields[0].dataType in [DoubleType(), FloatType()]:
        null_count = col_data.filter(col(columna).isNull() | isnan(col(columna))).count()
    else:
        null_count = col_data.filter(col(columna).isNull() | (col(columna) == "")).count()
    
    valid_count = num_total - null_count
    percent_valid = (valid_count / num_total) * 100
    percent_missing = (null_count / num_total) * 100
    
    # Valores únicos
    unique_count = col_data.distinct().count()
    
    # Valor más común
    most_common_row = col_data.groupBy(columna).count().orderBy(desc("count")).first()
    if most_common_row and most_common_row[0] is not None:
        most_common_value = most_common_row[0]
        most_common_percent = (most_common_row[1] / num_total) * 100
    else:
        most_common_value = None
        most_common_percent = 0
    
    # Mostrar resultados básicos
    print(f"Valid: {valid_count:,} ({percent_valid:.2f}%)")
    print(f"Missing: {null_count:,} ({percent_missing:.2f}%)")
    print(f"Unique values: {unique_count:,}")
    print(f"Most Common: {most_common_value} ({most_common_percent:.2f}%)")
    print(f"Mismatched: 0 (0%)")  # No hacemos validaciones externas

    # Estadísticas para valores numericos 
    tipo = col_data.schema.fields[0].dataType
    if isinstance(tipo, (DoubleType, FloatType, IntegerType, LongType)):
        stats = col_data.describe().toPandas().set_index('summary')
        print("\nEstadísticas numéricas:")
        for metric in ["mean", "stddev", "min", "max"]:
            value = stats.loc[metric, columna]
            print(f"{metric.capitalize()}: {value}")

        # Quartiles 
        approx = col_data.approxQuantile(columna, [0.25, 0.5, 0.75], 0.01)
        print(f"25%: {approx[0]}")
        print(f"50% (Median): {approx[1]}")
        print(f"75%: {approx[2]}")

# -------------------------------------------
# Aplicar análisis a todas las columnas
for col_name in df.columns:
    analizar_columna(df, col_name, num_total)



Número de columnas: 10
Número de registros: 3000000

Tipos de datos de cada columna:
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)




Número de registros con al menos un valor faltante: 2585451



Columnas con valores faltantes:
- Title
- Price
- User_id
- profileName
- review/helpfulness
- review/score
- review/time
- review/summary
- review/text

--- Columna: Id ---


Valid: 3,000,000 (100.00%)
Missing: 0 (0.00%)
Unique values: 221,998
Most Common: B000IEZE3G (0.23%)
Mismatched: 0 (0%)

--- Columna: Title ---


Valid: 2,999,792 (99.99%)
Missing: 208 (0.01%)
Unique values: 212,400
Most Common: The Hobbit (0.73%)
Mismatched: 0 (0%)

--- Columna: Price ---


...

Valid: 2,999,870 (100.00%)
Missing: 130 (0.00%)
Unique values: 2,038
Most Common: 5.0 (59.86%)
Mismatched: 0 (0%)

--- Columna: review/time ---


Valid: 2,999,973 (100.00%)
Missing: 27 (0.00%)
Unique values: 6,831
Most Common: 5.0 (0.28%)
Mismatched: 0 (0%)

--- Columna: review/summary ---


Valid: 2,999,935 (100.00%)
Missing: 65 (0.00%)
Unique values: 1,585,313
Most Common: Great Book (0.23%)
Mismatched: 0 (0%)

--- Columna: review/text ---


[Stage 261:======================================>                (14 + 6) / 20]

Valid: 2,999,957 (100.00%)
Missing: 43 (0.00%)
Unique values: 2,059,450
Most Common: 5.0 (0.01%)
Mismatched: 0 (0%)
